In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [23]:
!pip install tensorflow
!pip install keras
!pip install opencv-python
!pip install lxml

In [24]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.models import Model

def build_faster_rcnn_model(num_classes, input_shape):
    # Feature extraction backbone (ResNet50)
    backbone = ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')
    backbone.trainable = False

    # Region Proposal Network (RPN)
    rpn_conv = Conv2D(256, (3, 3), padding='same', activation='relu', name='rpn_conv')(backbone.output)
    rpn_cls_output = Conv2D(2, (1, 1), activation='softmax', name='rpn_cls_output')(rpn_conv)
    rpn_reg_output = Conv2D(4, (1, 1), activation='linear', name='rpn_reg_output')(rpn_conv)

    # RoI pooling layer (not shown in this example, you can implement it based on your requirement)

    # RoI classification and regression heads (not shown in this example, you can implement it based on your requirement)

    model = Model(inputs=backbone.input, outputs=[rpn_cls_output, rpn_reg_output])
    return model


In [31]:
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf

class CustomDataLoader(tf.keras.utils.Sequence):
    def __init__(self, image_paths, annotation_paths, batch_size, target_size):
        self.image_paths = image_paths
        self.annotation_paths = annotation_paths
        self.batch_size = batch_size
        self.target_size = target_size

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_annotation_paths = self.annotation_paths[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_images = []
        batch_bboxes = []

        for image_path, annotation_path in zip(batch_image_paths, batch_annotation_paths):
            image = cv2.imread(image_path)
            image = cv2.resize(image, self.target_size)
            batch_images.append(image)

            # Parse the XML annotation file to extract bounding box coordinates
            tree = ET.parse(annotation_path)
            root = tree.getroot()
            bboxes = []
            for obj in root.findall('object'):
                xmin = int(obj.find('bndbox').find('xmin').text)
                ymin = int(obj.find('bndbox').find('ymin').text)
                xmax = int(obj.find('bndbox').find('xmax').text)
                ymax = int(obj.find('bndbox').find('ymax').text)
                bboxes.append([xmin, ymin, xmax, ymax])

            # Convert bounding box coordinates to TensorFlow tensor
            batch_bboxes.append(tf.convert_to_tensor(bboxes, dtype=tf.float32))

        # Pad bounding boxes with zeros to ensure uniform shape
        batch_bboxes_padded = tf.keras.preprocessing.sequence.pad_sequences(batch_bboxes, padding='post')

        batch_images = np.array(batch_images)
        batch_bboxes_padded = tf.convert_to_tensor(batch_bboxes_padded)

        return batch_images, batch_bboxes_padded


In [32]:
# Replace '/content/dataset/images' and '/content/dataset/annotations' with the paths to your image and annotation directories
image_paths = [os.path.join('/content/drive/MyDrive/rcnn/images', filename) for filename in os.listdir('/content/drive/MyDrive/rcnn/images') if filename.endswith('.jpg')]
annotation_paths = [os.path.join('/content/drive/MyDrive/rcnn/annotations', filename) for filename in os.listdir('/content/drive/MyDrive/rcnn/annotations') if filename.endswith('.xml')]
batch_size = 4
target_size = (224, 224)

data_loader = CustomDataLoader(image_paths, annotation_paths, batch_size, target_size)


In [40]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


In [41]:
class SmoothL1Loss(tf.keras.losses.Loss):
    def __init__(self, delta=1.0, name='smooth_l1_loss'):
        super(SmoothL1Loss, self).__init__(name=name)
        self.delta = delta

    def call(self, y_true, y_pred):
        x = tf.abs(y_true - y_pred)
        return tf.where(x < self.delta, 0.5 * x ** 2 / self.delta, x - 0.5 * self.delta)


In [44]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Define your custom Faster R-CNN model
def custom_faster_rcnn():
    # Define the input layer
    inputs = Input(shape=(224, 224, 3))

    # Add convolutional and pooling layers (replace this with your custom architecture)
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    return model

# Create an instance of your custom Faster R-CNN model
model = custom_faster_rcnn()


In [45]:
smooth_l1_loss = SmoothL1Loss()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'regression': smooth_l1_loss},
              metrics={})


In [46]:
# Assuming 'data_loader' is your custom data loader
model.fit(data_loader, epochs=10)


Epoch 1/10


ValueError: ignored

In [38]:
import tensorflow as tf

class SmoothL1Loss(tf.keras.losses.Loss):
    def __init__(self, delta=1.0, name='smooth_l1_loss'):
        super(SmoothL1Loss, self).__init__(name=name)
        self.delta = delta

    def call(self, y_true, y_pred):
        x = tf.abs(y_true - y_pred)
        return tf.where(x < self.delta, 0.5 * x ** 2 / self.delta, x - 0.5 * self.delta)

# Create an instance of the custom loss function
smooth_l1_loss = SmoothL1Loss()

# Now you can use 'smooth_l1_loss' as your loss function in model compilation


In [39]:
num_classes = 5 # Replace with the number of classes in your custom dataset
input_shape = (target_size[0], target_size[1], 3)

model = build_faster_rcnn_model(num_classes, input_shape)
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'smooth_l1'])

# Start training with your custom dataset
model.fit(data_loader, epochs=10)


Epoch 1/10


ValueError: ignored